### 문제 2-3: 학생 정보 구조화 시스템

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field
from typing import List

In [25]:
# Pydantic 모델 정의
class PydanticModel(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="목표")

In [26]:
# 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=PydanticModel)

In [27]:
template="""
학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 
    구조화된 형태로 추출하는 시스템을 만드세요.
다음 요청을 분석하여 
이름, 나이, 전공, 취미 리스트, 목표
로 나눠 출력형식에 맞춰 JSON으로 추출하세요. 

요청: {query}
출력 형식: {format_instructions}
"""

In [28]:
prompt = ChatPromptTemplate.from_template(template=template)

In [29]:
prompt = prompt.partial(format_instructions = parser.get_format_instructions())

In [30]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

In [31]:
query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
result = chain.invoke({"query": query})


In [35]:
print(f"""{{
    "name": {result.name}, 
    "age": {result.age}, 
    "major": {result.major},
    "hobbies": {result.hobbies},
    "goal": {result.goal}
}}""")

{
    "name": 김민수, 
    "age": 22, 
    "major": 컴퓨터공학,
    "hobbies": ['게임하기', '영화보기', '코딩'],
    "goal": 훌륭한 개발자가 되는 것
}
